<a href="https://colab.research.google.com/github/edquestofficial/Gen-AI-Cohort/blob/main/2024/april/Level_2/LLaMA_Index/LLaMA_Index_Text_to_SQL_with_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to SQL with Semantic Search

## Reference

* [YouTube](https://www.youtube.com/watch?v=ZIvcVJGtCrY&t)
* [Notebook](https://www.llamaindex.ai/blog/combining-text-to-sql-with-semantic-search-for-retrieval-augmented-generation-c60af30ec3b)

In [54]:
%pip install llama-index-vector-stores-pinecone
%pip install llama-index-readers-wikipedia
%pip install llama-index-llms-openai

In [55]:
! pip install llama-index

In [ ]:
# install wikipedia python package
! pip install wikipedia

## Mount Google Drive

In [57]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [58]:
import os

secret_base_path = "/content/drive/MyDrive/Gen AI Course/secrets"
gemini_api_key_path = f"{secret_base_path}/gemini_api_key.txt"
openai_api_key_path = f"{secret_base_path}/openai_api_key.txt"
pinecone_api_key_path = f"{secret_base_path}/pinecone_api_key.txt"

with open(gemini_api_key_path, "r") as f:
  gemini_api_key = ' '.join(f.readlines())
  os.environ["GOOGLE_API_KEY"] = gemini_api_key

with open(openai_api_key_path, "r") as f:
  openai_api_key = ' '.join(f.readlines())

with open(pinecone_api_key_path, "r") as f:
  pinecone_api_key = ' '.join(f.readlines())

In [59]:
os.environ["PINECONE_API_KEY"] = pinecone_api_key

In [60]:
import openai
import os

os.environ["OPENAI_API_KEY"] = openai_api_key

In [61]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [64]:
# define pinecone index
# Reference: https://app.pinecone.io/organizations/-NiO4QwDpIIf4REEjxr7/projects/9f76d077-1774-499b-8745-8f82466dc7a1/indexes
import pinecone
import os
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

index_name = "quickstart"
pc.create_index(
    name=index_name,
    dimension=1536, # Replace with your model dimensions
    metric="euclidean", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [63]:
# pc.delete_index(index_name)

In [65]:
pinecone_index = pc.Index("quickstart")

In [66]:
from llama_index.core import StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex


# define pinecone vector index
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index, namespace="wiki_cities"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context)

## Create Database Schema + Test Data

In [67]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [68]:
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [69]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [70]:
# print tables
metadata_obj.tables.keys()

dict_keys(['city_stats'])

In [71]:
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [72]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Berlin', 3645000, 'Germany')]


## Load Data into Vector DB (Pinecone)

In [73]:
from llama_index.readers.wikipedia import WikipediaReader

cities = ["Toronto", "Berlin", "Tokyo"]
wiki_docs = WikipediaReader().load_data(pages=cities)

## Build SQL Index

In [74]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [75]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
)

## Build Vector Index

In [76]:
from llama_index.core import Settings

# Insert documents into vector index
# Each document has metadata of the city attached
for city, wiki_doc in zip(cities, wiki_docs):
    nodes = Settings.node_parser.get_nodes_from_documents([wiki_doc])
    # add metadata to each node
    for node in nodes:
        node.metadata = {"title": city}
        # print(node)
    vector_index.insert_nodes(nodes)

Upserted vectors:   0%|          | 0/21 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/21 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/14 [00:00<?, ?it/s]

## Define Query Engines, Set as Tools

In [77]:
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo
from llama_index.core.query_engine import RetrieverQueryEngine


vector_store_info = VectorStoreInfo(
    content_info="articles about different cities",
    metadata_info=[
        MetadataInfo(
            name="title", type="str", description="The name of the city"
        ),
    ],
)
vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, llm=OpenAI(model="gpt-4")
)

In [78]:
from llama_index.core.tools import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table containing: city_stats, containing the population/country of"
        " each city"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    description=(
        f"Useful for answering semantic questions about different cities"
    ),
)

## Define SQLAutoVectorQueryEngine

In [47]:
from llama_index.core.query_engine import SQLAutoVectorQueryEngine

query_engine = SQLAutoVectorQueryEngine(
    sql_tool, vector_tool, llm=OpenAI(model="gpt-3.5-turbo")
)

In [48]:
response = query_engine.query(
    "Tell me about the arts and culture of the city with the highest population"
)

Querying SQL database: The choice is relevant for translating a natural language query into a SQL query over a table containing city_stats, which includes population data. This choice can help retrieve information about the city with the highest population.
SQL query: SELECT city_name, population, country FROM city_stats ORDER BY population DESC LIMIT 1;
SQL response: Tokyo, Japan has a population of 13.96 million people, making it the city with the highest population. The arts and culture of Tokyo are rich and diverse, with a mix of traditional Japanese arts such as tea ceremonies, kabuki theater, and sumo wrestling, as well as modern influences like anime, manga, and contemporary art. The city is home to numerous museums, galleries, theaters, and cultural events that showcase both traditional and contemporary Japanese art forms. Additionally, Tokyo is known for its vibrant street art scene, fashion districts, and bustling nightlife, making it a hub of creativity and innovation in the

In [49]:
print(str(response))

Tokyo, Japan has a population of 13.96 million people, making it the city with the highest population. The arts and culture of Tokyo are rich and diverse, with a mix of traditional Japanese arts such as tea ceremonies, kabuki theater, and sumo wrestling, as well as modern influences like anime, manga, and contemporary art. The city is home to numerous museums, galleries, theaters, and cultural events that showcase both traditional and contemporary Japanese art forms. Additionally, Tokyo is known for its vibrant street art scene, fashion districts, and bustling nightlife, making it a hub of creativity and innovation in the arts.


In [50]:
response = query_engine.query("Tell me about the history of Berlin")

Querying other query engine: The choice is relevant for answering semantic questions about different cities, which includes providing information about the history of a specific city like Berlin.
Query Engine response: Berlin's history is rich and complex, dating back to human settlements around 60,000 BC. The region came under German rule in the 12th century as part of the Margraviate of Brandenburg. The city suffered significant damage during the Thirty Years' War in the 17th century, but under the rule of Frederick William, it saw a period of immigration and religious tolerance. By 1700, about 30% of Berlin's residents were French due to Huguenot immigration. 

In 1701, Berlin became the capital of the newly formed Kingdom of Prussia. The 19th century brought the Industrial Revolution, transforming Berlin into a major railway hub and economic center of Germany. In 1871, it became the capital of the newly founded German Empire.

The early 20th century saw Berlin as a fertile ground f

In [51]:
print(str(response))

Berlin's history is rich and complex, dating back to human settlements around 60,000 BC. The region came under German rule in the 12th century as part of the Margraviate of Brandenburg. The city suffered significant damage during the Thirty Years' War in the 17th century, but under the rule of Frederick William, it saw a period of immigration and religious tolerance. By 1700, about 30% of Berlin's residents were French due to Huguenot immigration. 

In 1701, Berlin became the capital of the newly formed Kingdom of Prussia. The 19th century brought the Industrial Revolution, transforming Berlin into a major railway hub and economic center of Germany. In 1871, it became the capital of the newly founded German Empire.

The early 20th century saw Berlin as a fertile ground for the German Expressionist movement and a center of the Roaring Twenties. However, the city also experienced political unrest and economic uncertainties. In 1933, Adolf Hitler and the Nazi Party came to power, leading 

In [52]:
response = query_engine.query(
    "Can you give me the country corresponding to each city?"
)

Querying SQL database: This choice is useful for translating a natural language query into a SQL query over a table containing city_stats, which includes the population/country of each city. This would allow for retrieving the country corresponding to each city.
SQL query: SELECT city_name, country
FROM city_stats
ORDER BY city_name;
SQL response: The corresponding countries for each city are Germany for Berlin, Japan for Tokyo, and Canada for Toronto.
Transformed query given SQL response: None


In [53]:
print(str(response))

The corresponding countries for each city are Germany for Berlin, Japan for Tokyo, and Canada for Toronto.
